# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f98645210d0>
├── 'a' --> tensor([[-0.6242, -0.1806, -0.5731],
│                   [ 0.2112, -0.0425, -2.4086]])
└── 'x' --> <FastTreeValue 0x7f9864521a30>
    └── 'c' --> tensor([[-0.0767,  0.6759,  2.2328, -0.8306],
                        [ 0.3424, -2.7923, -0.1039,  2.1747],
                        [-0.3501,  1.7121, -0.4480, -3.2059]])

In [4]:
t.a

tensor([[-0.6242, -0.1806, -0.5731],
        [ 0.2112, -0.0425, -2.4086]])

In [5]:
%timeit t.a

98.7 ns ± 0.0809 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f98645210d0>
├── 'a' --> tensor([[ 0.5328,  0.2767, -0.2272],
│                   [ 0.7736, -0.6992, -2.0014]])
└── 'x' --> <FastTreeValue 0x7f9864521a30>
    └── 'c' --> tensor([[-0.0767,  0.6759,  2.2328, -0.8306],
                        [ 0.3424, -2.7923, -0.1039,  2.1747],
                        [-0.3501,  1.7121, -0.4480, -3.2059]])

In [7]:
%timeit t.a = new_value

103 ns ± 0.132 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.6242, -0.1806, -0.5731],
               [ 0.2112, -0.0425, -2.4086]]),
    x: Batch(
           c: tensor([[-0.0767,  0.6759,  2.2328, -0.8306],
                      [ 0.3424, -2.7923, -0.1039,  2.1747],
                      [-0.3501,  1.7121, -0.4480, -3.2059]]),
       ),
)

In [10]:
b.a

tensor([[-0.6242, -0.1806, -0.5731],
        [ 0.2112, -0.0425, -2.4086]])

In [11]:
%timeit b.a

79.6 ns ± 0.0716 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.9105, -1.2043,  0.2100],
               [ 0.5418,  0.3174,  0.1394]]),
    x: Batch(
           c: tensor([[-0.0767,  0.6759,  2.2328, -0.8306],
                      [ 0.3424, -2.7923, -0.1039,  2.1747],
                      [-0.3501,  1.7121, -0.4480, -3.2059]]),
       ),
)

In [13]:
%timeit b.a = new_value

661 ns ± 0.977 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.06 µs ± 4.05 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12.6 µs ± 25.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

174 µs ± 571 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

175 µs ± 994 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f977ac0ce50>
├── 'a' --> tensor([[[-0.6242, -0.1806, -0.5731],
│                    [ 0.2112, -0.0425, -2.4086]],
│           
│                   [[-0.6242, -0.1806, -0.5731],
│                    [ 0.2112, -0.0425, -2.4086]],
│           
│                   [[-0.6242, -0.1806, -0.5731],
│                    [ 0.2112, -0.0425, -2.4086]],
│           
│                   [[-0.6242, -0.1806, -0.5731],
│                    [ 0.2112, -0.0425, -2.4086]],
│           
│                   [[-0.6242, -0.1806, -0.5731],
│                    [ 0.2112, -0.0425, -2.4086]],
│           
│                   [[-0.6242, -0.1806, -0.5731],
│                    [ 0.2112, -0.0425, -2.4086]],
│           
│                   [[-0.6242, -0.1806, -0.5731],
│                    [ 0.2112, -0.0425, -2.4086]],
│           
│                   [[-0.6242, -0.1806, -0.5731],
│                    [ 0.2112, -0.0425, -2.4086]]])
└── 'x' --> <FastTreeValue 0x7f977ac0ce20>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

39.5 µs ± 148 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f977ac0cac0>
├── 'a' --> tensor([[-0.6242, -0.1806, -0.5731],
│                   [ 0.2112, -0.0425, -2.4086],
│                   [-0.6242, -0.1806, -0.5731],
│                   [ 0.2112, -0.0425, -2.4086],
│                   [-0.6242, -0.1806, -0.5731],
│                   [ 0.2112, -0.0425, -2.4086],
│                   [-0.6242, -0.1806, -0.5731],
│                   [ 0.2112, -0.0425, -2.4086],
│                   [-0.6242, -0.1806, -0.5731],
│                   [ 0.2112, -0.0425, -2.4086],
│                   [-0.6242, -0.1806, -0.5731],
│                   [ 0.2112, -0.0425, -2.4086],
│                   [-0.6242, -0.1806, -0.5731],
│                   [ 0.2112, -0.0425, -2.4086],
│                   [-0.6242, -0.1806, -0.5731],
│                   [ 0.2112, -0.0425, -2.4086]])
└── 'x' --> <FastTreeValue 0x7f977ac0c760>
    └── 'c' --> tensor([[-0.0767,  0.6759,  2.2328, -0.8306],
                        [ 0.3424, -2.7923, -0.1039,  2.1747],
                 

In [23]:
%timeit t_cat(trees)

37.6 µs ± 516 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

75.1 µs ± 39.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.0767,  0.6759,  2.2328, -0.8306],
                       [ 0.3424, -2.7923, -0.1039,  2.1747],
                       [-0.3501,  1.7121, -0.4480, -3.2059]],
              
                      [[-0.0767,  0.6759,  2.2328, -0.8306],
                       [ 0.3424, -2.7923, -0.1039,  2.1747],
                       [-0.3501,  1.7121, -0.4480, -3.2059]],
              
                      [[-0.0767,  0.6759,  2.2328, -0.8306],
                       [ 0.3424, -2.7923, -0.1039,  2.1747],
                       [-0.3501,  1.7121, -0.4480, -3.2059]],
              
                      [[-0.0767,  0.6759,  2.2328, -0.8306],
                       [ 0.3424, -2.7923, -0.1039,  2.1747],
                       [-0.3501,  1.7121, -0.4480, -3.2059]],
              
                      [[-0.0767,  0.6759,  2.2328, -0.8306],
                       [ 0.3424, -2.7923, -0.1039,  2.1747],
                       [-0.3501,  1.7121, -0.4480, -3.2059]],

In [26]:
%timeit Batch.stack(batches)

98.8 µs ± 206 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.0767,  0.6759,  2.2328, -0.8306],
                      [ 0.3424, -2.7923, -0.1039,  2.1747],
                      [-0.3501,  1.7121, -0.4480, -3.2059],
                      [-0.0767,  0.6759,  2.2328, -0.8306],
                      [ 0.3424, -2.7923, -0.1039,  2.1747],
                      [-0.3501,  1.7121, -0.4480, -3.2059],
                      [-0.0767,  0.6759,  2.2328, -0.8306],
                      [ 0.3424, -2.7923, -0.1039,  2.1747],
                      [-0.3501,  1.7121, -0.4480, -3.2059],
                      [-0.0767,  0.6759,  2.2328, -0.8306],
                      [ 0.3424, -2.7923, -0.1039,  2.1747],
                      [-0.3501,  1.7121, -0.4480, -3.2059],
                      [-0.0767,  0.6759,  2.2328, -0.8306],
                      [ 0.3424, -2.7923, -0.1039,  2.1747],
                      [-0.3501,  1.7121, -0.4480, -3.2059],
                      [-0.0767,  0.6759,  2.2328, -0.8306],
                   

In [28]:
%timeit Batch.cat(batches)

183 µs ± 416 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

467 µs ± 16.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
